<a href="https://colab.research.google.com/github/Mays-Waddah/test/blob/main/AI2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# قراءة ملف CSV
df = pd.read_csv('university data.csv')

# عرض أول 5 صفوف لفهم شكل البيانات
print("أول 5 صفوف من البيانات:")
print(df.head())

# عرض معلومات عن الأعمدة وأنواع البيانات والقيم المفقودة
print("\nمعلومات عن الأعمدة وأنواع البيانات:")
df.info()

أول 5 صفوف من البيانات:
      userName                       major  researchExp  industryExp  \
0       143saf         Systems and Control            0           18   
1   7790ashish   Manufacturing Engineering            0            0   
2         AB25  (MIS / MSIM / MSIS / MSIT)            0           66   
3     abhijitg                         NaN            0            0   
4  abhijitgang                         MIS            0            0   

  specialization  toeflScore program                 department toeflEssay  \
0       Robotics       112.0      MS  Instrumentation & Control         26   
1            NaN         NaN      MS                          0        NaN   
2            NaN        94.0      MS       Computer Engineering         21   
3            NaN         NaN     NaN                          0        NaN   
4            NaN        81.0      MS                   computer        NaN   

   internExp  ...  termAndYear  confPubs                    ugCollege gmat

In [2]:
!pip install scikit-fuzzy matplotlib numpy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 11.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np

# Assuming 'university data.csv' is uploaded to Colab's session storage
df = pd.read_csv('university data.csv')

# تنظيف البيانات وتوحيد CGPA (إذا لزم الأمر)
# يمكن أن يكون CGPA على مقياس 10 أو 100 أو 4.0. سنقوم بتوحيده إلى 100 إذا كانت cgpaScale موجودة ومختلفة.
def normalize_cgpa(row):
    if row['cgpaScale'] == 0: # Handle cases where cgpaScale is 0 to avoid division by zero
        return 0
    return (row['cgpa'] / row['cgpaScale']) * 100

# Apply normalization, handling potential NaN or invalid values
df['normalized_cgpa'] = df.apply(normalize_cgpa, axis=1)
df['normalized_cgpa'].replace([np.inf, -np.inf], np.nan, inplace=True)
df['normalized_cgpa'].fillna(df['normalized_cgpa'].mean(), inplace=True) # Fill NaNs for safety

# قم بتنظيف toeflScore و researchExp من القيم المفقودة أو غير الصحيحة
df['toeflScore'].fillna(df['toeflScore'].mean(), inplace=True)
df['researchExp'].fillna(df['researchExp'].mode()[0], inplace=True) # mode for int, mean for float

# يمكنك طباعة إحصائيات لمساعدتك في تحديد النطاقات بدقة
print("\nإحصائيات المدخلات المختارة:")
print(df[['normalized_cgpa', 'toeflScore', 'researchExp']].describe())


إحصائيات المدخلات المختارة:
       normalized_cgpa    toeflScore   researchExp
count     53644.000000  53644.000000  53644.000000
mean         73.625277    109.558826      0.339535
std          34.445443     34.344806      2.602942
min           0.000000      0.000000      0.000000
25%          68.200000    101.000000      0.000000
50%          76.000000    107.000000      0.000000
75%          83.600000    111.000000      0.000000
max        1825.000000   1350.000000     53.000000


<ipython-input-3-3348082644>:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['normalized_cgpa'].replace([np.inf, -np.inf], np.nan, inplace=True)
<ipython-input-3-3348082644>:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplac

In [4]:
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt

# 1. تعريف المتغيرات الضبابية (Inputs and Output)
# بناءً على النطاقات التي حددناها
cgpa = ctrl.Antecedent(np.arange(0, 101, 1), 'cgpa')
toeflScore = ctrl.Antecedent(np.arange(0, 121, 1), 'toeflScore')
researchExp = ctrl.Antecedent(np.arange(0, 51, 1), 'researchExp')
admission_likelihood = ctrl.Consequent(np.arange(0, 101, 1), 'admission_likelihood')

# 2. تعريف دوال الانتماء (Membership Functions) - 5 مجموعات لكل
# CGPA
cgpa['very_low'] = fuzz.trimf(cgpa.universe, [0, 0, 40])
cgpa['low'] = fuzz.trimf(cgpa.universe, [0, 40, 60])
cgpa['medium'] = fuzz.trimf(cgpa.universe, [40, 60, 80])
cgpa['high'] = fuzz.trimf(cgpa.universe, [60, 80, 100])
cgpa['very_high'] = fuzz.trimf(cgpa.universe, [80, 100, 100])

# TOEFL Score
toeflScore['very_low'] = fuzz.trimf(toeflScore.universe, [0, 0, 70])
toeflScore['low'] = fuzz.trimf(toeflScore.universe, [0, 70, 85])
toeflScore['medium'] = fuzz.trimf(toeflScore.universe, [70, 85, 100])
toeflScore['high'] = fuzz.trimf(toeflScore.universe, [85, 100, 115])
toeflScore['very_high'] = fuzz.trimf(toeflScore.universe, [100, 120, 120])

# Research Experience
researchExp['none'] = fuzz.trimf(researchExp.universe, [0, 0, 10])
researchExp['low'] = fuzz.trimf(researchExp.universe, [0, 10, 20])
researchExp['medium'] = fuzz.trimf(researchExp.universe, [10, 20, 30])
researchExp['high'] = fuzz.trimf(researchExp.universe, [20, 30, 40])
researchExp['very_high'] = fuzz.trimf(researchExp.universe, [30, 50, 50])

# Admission Likelihood (Output)
admission_likelihood['very_low'] = fuzz.trimf(admission_likelihood.universe, [0, 0, 25])
admission_likelihood['low'] = fuzz.trimf(admission_likelihood.universe, [0, 25, 50])
admission_likelihood['medium'] = fuzz.trimf(admission_likelihood.universe, [25, 50, 75])
admission_likelihood['high'] = fuzz.trimf(admission_likelihood.universe, [50, 75, 100])
admission_likelihood['very_high'] = fuzz.trimf(admission_likelihood.universe, [75, 100, 100])

# (اختياري) لرسم دوال الانتماء للتأكد منها
# cgpa.view()
# toeflScore.view()
# researchExp.view()
# admission_likelihood.view()
# plt.show()

# 3. تعريف القواعد (Rules)
# بناءً على القواعد التي وضعناها يدوياً، مع إضافة المزيد لتغطية السيناريوهات
rule1 = ctrl.Rule(cgpa['very_high'] & toeflScore['very_high'], admission_likelihood['very_high'])
rule2 = ctrl.Rule(cgpa['low'] | toeflScore['low'], admission_likelihood['very_low'])
rule3 = ctrl.Rule(cgpa['medium'] & researchExp['medium'], admission_likelihood['medium'])
rule4 = ctrl.Rule(cgpa['high'] & researchExp['high'], admission_likelihood['high'])
rule5 = ctrl.Rule(toeflScore['very_low'], admission_likelihood['very_low'])
rule6 = ctrl.Rule(cgpa['very_high'] & researchExp['very_high'], admission_likelihood['very_high'])
rule7 = ctrl.Rule(cgpa['low'] & toeflScore['medium'], admission_likelihood['low'])
# أضف المزيد من القواعد هنا بناءً على تحليلك وفهمك لبياناتك

# 4. بناء نظام التحكم الضبابي (Fuzzy Control System)
admission_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7]) # أضف جميع القواعد
admission_simulation = ctrl.ControlSystemSimulation(admission_ctrl)

# 5. إدخال قيم الحالة الجديدة (المثال اليدوي الذي طبقناه)
admission_simulation.input['cgpa'] = 70
admission_simulation.input['toeflScore'] = 90
admission_simulation.input['researchExp'] = 25

# 6. حساب النتيجة (Compute)
admission_simulation.compute()

# 7. عرض النتيجة
print(f"Predicted Admission Likelihood: {admission_simulation.output['admission_likelihood']:.2f}")

# (اختياري) لعرض المخرجات الضبابية بعد الحساب
# admission_likelihood.view(sim=admission_simulation)
# plt.show()

Predicted Admission Likelihood: 62.50
